In [13]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import box
import shapely.wkt
from shapely.wkt import loads
from tqdm import tqdm
import pandas as pd
import os
import sys
import numpy as np
import glob
from solaris.vector.polygon import convert_poly_coords
# from solaris.vector.mask import mask_to_poly_geojson
from solaris.utils.geo import get_crs
from osgeo import gdal

In [23]:
def non_max_suppression(boxes, probs=[], overlapThresh=0.5):

    print("Executing non-max suppression...")
    len_init = len(boxes)

    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return [], [], []

    # boxes_tot = boxes  # np.asarray(boxes)
    boxes = np.asarray([b[:4] for b in boxes])
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    # compute the area of the bounding boxes
    area = (x2 - x1) * (y2 - y1)

    # sort the boxes by the bottom-right y-coordinate of the bounding box
    if len(probs) == 0:
        idxs = np.argsort(((x2 - x1) * (y2 - y1)))
    # sort boxes by the highest prob (descending order)
    else:
        idxs = np.argsort(probs)[::-1]

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1)
        h = np.maximum(0, yy2 - yy1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have
        idxs = np.delete(
            idxs,
            np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    print("  non-max suppression init boxes:", len_init)
    print("  non-max suppression final boxes:", len(pick))
    return pick

def vectorize_yolo(df, output_dir, nms=True, overlapThresh=0.5):
    # Create directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    bb_dir = os.path.join(output_dir, "bounding_boxes")
    if not os.path.exists(bb_dir):
        os.makedirs(bb_dir)
    try:
        df['bb_geometry'] = str(df['bb_geometry']).apply(loads)
    except AttributeError:
        pass
    for img_id in df['image'].unique():
        img_df = df[df['image'] == img_id]
        img_df = img_df.reset_index()
        crs = str(img_df['crs'].iloc[0])
        if nms is True:
            confs = []
            bbs = []
            for bb_geom in img_df['bb_geometry']:
                bb_geom = [bb_geom.bounds[0],bb_geom.bounds[1],bb_geom.bounds[2],bb_geom.bounds[3]]
                bbs.append(bb_geom)
            b_box_nms_indices = non_max_suppression(bbs, confs, overlapThresh=overlapThresh)
            b_box_df_out = gpd.GeoDataFrame(img_df.loc[b_box_nms_indices], crs=crs, geometry=img_df.bb_geometry)
            b_box_df_out = b_box_df_out.drop(columns=["bb_geometry", "crs", 'index'])
        else:
            b_box_df_out = gpd.GeoDataFrame(img_df, crs=crs, geometry=img_df.bb_geometry)
            b_box_df_out = b_box_df_out.drop(columns=["bb_geometry", "crs", 'index'])
        output = os.path.join(bb_dir, img_id.split(".")[0] + ".geojson")
        b_box_df_out.to_file(output, driver='GeoJSON')

In [24]:
input_dir = "C:/Users/Professional/yoloplanes/yolo/inference/class_one_out"
output_dir = input_dir
image_dir = "C:/Users/Professional/yoloplanes/class_one/images/val"
lookup_table = "C:/Users/Professional/yoloplanes/geojsons_test/yolo_class_one/custom_class_lookup.csv"
lookup_table = pd.read_csv(lookup_table)
lookup_table['yolo'] = lookup_table['custom_id'] - 1
lookup_table.set_index('yolo')
print(lookup_table)

   Unnamed: 0 propulsion  custom_id  yolo
0           0        jet          1     0
1           1  propeller          2     1
2           2  unpowered          3     2


### Executing Post-Processing

The following cells convert the labels to bounding boxes and then also create a .csv from the dataframe with the detection results and execute the non-max supression on the predictions. 

In [28]:
def convert_reverse(size, box):
    '''
    From: https://github.com/CosmiQ/yolt/blob/master/scripts/convert.py
    Back out pixel coords from yolo format
    input = image_size (w,h), 
        box = [x,y,w,h]'''
    x,y,w,h = box
    dw = 1./size[0]
    dh = 1./size[1]
    w0 = w/dw
    h0 = h/dh
    xmid = x/dw
    ymid = y/dh
    x0, x1 = xmid - w0/2., xmid + w0/2.
    y0, y1 = ymid - h0/2., ymid + h0/2.
    return [x0, x1, y0, y1]

class_ids = []
classes = []
confidences = []
images = []
crs_s = []
bb_geoms = []
os.chdir(input_dir)
txts = glob.glob("*.txt")
for txt in tqdm(txts):
    image = txt.split(".")[0] + ".png"
    im_path = os.path.join(image_dir, image)
    image = image.split("_tile")[0]
    df = pd.read_csv(txt,header = None, sep = " ")
    for index, row in df.iterrows():
        output = convert_reverse((512,512), [row[1], row[2], row[3], row[4]])
        b_box = box(output[0], output[2], output[1], output[3])
        bb_geom = convert_poly_coords(b_box, raster_src=im_path)
        bb_geoms.append(bb_geom)
        crs = get_crs(gdal.Open(im_path))
        crs = str(crs)
        crs_s.append(crs)
        class_ids.append(row[0])
        confidences.append(row[4])
        images.append(image)
        classes.append(str(lookup_table['propulsion'][row[0]])) # change this based uppn the custom class
dict = {'class_id': class_ids, 'class': classes, 'confidence': confidences, 'image': images, 'crs': crs_s, 'bb_geometry': bb_geoms}
df = pd.DataFrame(dict)

100%|██████████| 2661/2661 [01:12<00:00, 36.82it/s]


In [29]:
df.to_csv(path_or_buf=os.path.join(output_dir, "detection_results.csv"))

In [30]:
vectorize_yolo(df, output_dir, nms=True, overlapThresh=0.5)

Executing non-max suppression...
  non-max suppression init boxes: 31
  non-max suppression final boxes: 21
Executing non-max suppression...
  non-max suppression init boxes: 105
  non-max suppression final boxes: 66
Executing non-max suppression...
  non-max suppression init boxes: 217
  non-max suppression final boxes: 136
Executing non-max suppression...
  non-max suppression init boxes: 69
  non-max suppression final boxes: 41
Executing non-max suppression...
  non-max suppression init boxes: 46
  non-max suppression final boxes: 26
Executing non-max suppression...
  non-max suppression init boxes: 165
  non-max suppression final boxes: 102
Executing non-max suppression...
  non-max suppression init boxes: 190
  non-max suppression final boxes: 98
Executing non-max suppression...
  non-max suppression init boxes: 46
  non-max suppression final boxes: 25
Executing non-max suppression...
  non-max suppression init boxes: 27
  non-max suppression final boxes: 20
Executing non-max supp